In [1]:
import pandas as pd
import matplotlib as plt
import seaborn as sns
import numpy as np

In [2]:
df = pd.read_csv('heart.csv').reset_index()

In [3]:
df.head()

,index,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 918 entries, 0 to 917
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   index           918 non-null    int64  
 1   Age             918 non-null    int64  
 2   Sex             918 non-null    object 
 3   ChestPainType   918 non-null    object 
 4   RestingBP       918 non-null    int64  
 5   Cholesterol     918 non-null    int64  
 6   FastingBS       918 non-null    int64  
 7   RestingECG      918 non-null    object 
 8   MaxHR           918 non-null    int64  
 9   ExerciseAngina  918 non-null    object 
 10  Oldpeak         918 non-null    float64
 11  ST_Slope        918 non-null    object 
 12  HeartDisease    918 non-null    int64  
dtypes: float64(1), int64(7), object(5)
memory usage: 93.4+ KB


In [5]:
df.describe()

,index,Age,RestingBP,Cholesterol,FastingBS,MaxHR,Oldpeak,HeartDisease
count,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000
mean,458.500000,53.510893,132.396514,198.799564,0.233115,136.809368,0.887364,0.553377
std,265.148072,9.432617,18.514154,109.384145,0.423046,25.460334,1.066570,0.497414
min,0.000000,28.000000,0.000000,0.000000,0.000000,60.000000,-2.600000,0.000000
25%,229.250000,47.000000,120.000000,173.250000,0.000000,120.000000,0.000000,0.000000
50%,458.500000,54.000000,130.000000,223.000000,0.000000,138.000000,0.600000,1.000000
75%,687.750000,60.000000,140.000000,267.000000,0.000000,156.000000,1.500000,1.000000
max,917.000000,77.000000,200.000000,603.000000,1.000000,202.000000,6.200000,1.000000


In [6]:
df['ChestPainType'].unique()

array(['ATA', 'NAP', 'ASY', 'TA'], dtype=object)

In [7]:
df['RestingECG'].unique()

array(['Normal', 'ST', 'LVH'], dtype=object)

In [8]:
df['ExerciseAngina'].unique()

array(['N', 'Y'], dtype=object)

In [9]:
df['ST_Slope'].unique()

array(['Up', 'Flat', 'Down'], dtype=object)

In [10]:
df_encode = pd.get_dummies(df[['ChestPainType','RestingECG', 'ST_Slope','ExerciseAngina','Sex' ]],prefix=['ChestPainType', 'RestingECG', 'ST_Slope', 'ExerciseAngina','Sex'], dummy_na=True).reset_index()

In [11]:
df = pd.merge(df,df_encode, on='index', how='left')

In [12]:
df.drop(columns=['ChestPainType', 'RestingECG', 'ExerciseAngina', 'ST_Slope', 'Sex'], inplace=True)

In [13]:
df.isnull().sum().sum()

0

In [32]:
X = df[['Age','RestingBP','Cholesterol','FastingBS','MaxHR','Oldpeak','ChestPainType_ASY','ChestPainType_ATA','ChestPainType_NAP','ChestPainType_nan','RestingECG_LVH','RestingECG_Normal' ,'RestingECG_ST','RestingECG_nan','ST_Slope_Down','ST_Slope_Flat','ST_Slope_Up','ST_Slope_nan','ExerciseAngina_N','ExerciseAngina_Y','ExerciseAngina_nan','Sex_F','Sex_M','Sex_nan']] 
y = df[['HeartDisease']].values.ravel()

In [34]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [75]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
model = LogisticRegression(max_iter=10000)


In [76]:
model.fit(X_train, y_train)

LogisticRegression(max_iter=10000)

In [77]:
y_pred = model.predict(X_test)

In [78]:
model.score(X_test, y_test)

0.8532608695652174

In [79]:
from sklearn.model_selection import cross_validate

In [80]:
cross_val_res = cross_validate(model,X,y, cv=10, scoring=['accuracy','recall','precision','f1'])

In [81]:
df_res = pd.DataFrame(cross_val_res)

In [82]:
df_res

,fit_time,score_time,test_accuracy,test_recall,test_precision,test_f1
0,0.149658,0.000000,0.836957,0.784314,0.909091,0.842105
1,0.171859,0.000000,0.923913,0.901961,0.958333,0.929293
2,0.218726,0.000000,0.869565,0.843137,0.914894,0.877551
3,0.286283,0.000000,0.956522,0.960784,0.960784,0.960784
4,0.217139,0.000000,0.869565,0.980392,0.819672,0.892857
5,0.201256,0.000000,0.847826,1.000000,0.784615,0.879310
6,0.303738,0.000000,0.880435,0.980392,0.833333,0.900901
7,0.134667,0.015623,0.836957,0.823529,0.875000,0.848485
8,0.046870,0.015623,0.736264,0.760000,0.760000,0.760000
9,0.093741,0.000000,0.780220,0.720000,0.857143,0.782609


In [100]:
from sklearn.model_selection import GridSearchCV
parameters = {'penalty': ['l1', 'l2'],'C': np.logspace(-4, 4, 20),
    'solver': ['liblinear']}
clf = GridSearchCV(model, param_grid=parameters,scoring = 'accuracy')

In [101]:
search_1 = clf.fit(X_train, y_train)
y_pred_grid = clf.predict(X_test)
search_1.best_params_

{'C': 0.615848211066026, 'penalty': 'l1', 'solver': 'liblinear'}

In [102]:
cross_val_res_n = cross_validate(clf,X,y, cv=10, scoring=['accuracy','recall','precision','f1'])

In [103]:
df_res_n = pd.DataFrame(cross_val_res_n)

In [104]:
df_res_n

,fit_time,score_time,test_accuracy,test_recall,test_precision,test_f1
0,1.064394,0.004364,0.826087,0.764706,0.906977,0.829787
1,1.069415,0.000000,0.913043,0.901961,0.938776,0.920000
2,1.066130,0.000000,0.869565,0.843137,0.914894,0.877551
3,1.089372,0.000000,0.945652,0.941176,0.960000,0.950495
4,1.056794,0.015559,0.869565,0.980392,0.819672,0.892857
5,1.028463,0.000000,0.847826,1.000000,0.784615,0.879310
6,1.039183,0.008021,0.880435,0.960784,0.844828,0.899083
7,1.122707,0.002732,0.836957,0.823529,0.875000,0.848485
8,1.161517,0.015633,0.747253,0.760000,0.775510,0.767677
9,1.036516,0.000000,0.791209,0.720000,0.878049,0.791209


In [105]:
from sklearn.metrics import accuracy_score,recall_score,precision_score,f1_score

print('Accuracy Score : ' + str(accuracy_score(y_test,y_pred)))
print('Precision Score : ' + str(precision_score(y_test,y_pred)))
print('Recall Score : ' + str(recall_score(y_test,y_pred)))
print('F1 Score : ' + str(f1_score(y_test,y_pred)))

from sklearn.metrics import confusion_matrix
print('Confusion Matrix : \n' + str(confusion_matrix(y_test,y_pred)))

Accuracy Score : 0.8532608695652174
Precision Score : 0.9
Recall Score : 0.8411214953271028
F1 Score : 0.8695652173913043
Confusion Matrix : 
[[67 10]
 [17 90]]


In [106]:
print('Accuracy Score : ' + str(accuracy_score(y_test,y_pred_grid)))
print('Precision Score : ' + str(precision_score(y_test,y_pred_grid)))
print('Recall Score : ' + str(recall_score(y_test,y_pred_grid)))
print('F1 Score : ' + str(f1_score(y_test,y_pred_grid)))
print('Confusion Matrix : \n' + str(confusion_matrix(y_test,y_pred_grid)))


Accuracy Score : 0.8532608695652174
Precision Score : 0.9081632653061225
Recall Score : 0.8317757009345794
F1 Score : 0.8682926829268293
Confusion Matrix : 
[[68  9]
 [18 89]]


In [115]:
from sklearn.model_selection import RandomizedSearchCV
param = {'solver' : ['newton-cg', 'lbfgs','sag', 'liblinear', 'saga'],'C' : np.logspace(-4, 4, 20)}
clf_1 = RandomizedSearchCV(model, param)
search = clf_1.fit(X,y)
search.best_params_


{'solver': 'newton-cg', 'C': 206.913808111479}

In [116]:
y_pred_rand = clf_1.predict(X_test)

In [117]:
print('Accuracy Score : ' + str(accuracy_score(y_test,y_pred_rand)))
print('Precision Score : ' + str(precision_score(y_test,y_pred_rand)))
print('Recall Score : ' + str(recall_score(y_test,y_pred_rand)))
print('F1 Score : ' + str(f1_score(y_test,y_pred_rand)))
print('Confusion Matrix : \n' + str(confusion_matrix(y_test,y_pred_rand)))


Accuracy Score : 0.8641304347826086
Precision Score : 0.9019607843137255
Recall Score : 0.8598130841121495
F1 Score : 0.8803827751196173
Confusion Matrix : 
[[67 10]
 [15 92]]


### Выводы
В случае bagging результат получился лучше
